#### Plot the final data that has been filtered by the rule_base_filtering.py with hovering labels, on click it will save the index and also add the marker to the map

In [7]:
from ipyleaflet import Map, GeoJSON, WidgetControl, TileLayer, Marker, Icon
from ipywidgets import HTML, Layout
import json
import threading
import time
import random

# Load your GeoJSON data
with open('../output/nebraska_final_data.geojson') as f:
    geojson_data = json.load(f)
for i, feature in enumerate(geojson_data['features']):
    # Assign an 'index' property to each feature
    feature['properties']['index'] = i
# Initialize the map
m = Map(center=(34.9, -92.3), zoom=6.5)
# Widget to display feature information on the map
widget = HTML()
widget.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=widget, position='bottomright')
m.add_control(control)
# Global variable to keep track of hover state
is_hovering = False
is_clicking = False
false_positives = []
# Define a hover handler
def handle_hover(event, feature, **kwargs):
    global is_hovering
    is_hovering = True
    properties = feature['properties']
    geometry = feature['geometry']['coordinates'][0][0]
    longitude, latitude = geometry[:2]
    index = feature['properties']['index']
    # Update the widget to show information
    widget.value = f'Index: {index}, Lat: {latitude}, Lon: {longitude}'
def handle_click(event, feature, **kwargs):
    global is_hovering, false_positives, m
    is_clicking = True
    index = feature['properties']['index']
    false_positives.append(index)
    print(f'Feature Index: {index}')
    longitude, latitude = feature['geometry']['coordinates'][0][0][:2]
    center = (latitude, longitude)
    marker = Marker(location=center, draggable=False)
    m.add(marker)


# Function to clear the widget if not hovering
def clear_widget(*args, **kwargs):
    global is_hovering
    if not is_hovering:
        widget.value = ''
    is_hovering = False

def change_click_color(feature):
    return {
        'color': 'blue',
        'fillColor': 'blue'
    }


# Create the GeoJSON layer with hover functionality and add it to the map
geo_json = GeoJSON(
    data=geojson_data,
    style={'color': 'yellow', 'fillcolor': 'yellow', 'opacity': 0.5},
    hover_style={'color': 'green', 'fillOpacity': 0.8}
    
)
geo_json.on_click(handle_click)
geo_json.on_hover(handle_hover)


esri_satellite = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
m.add_layer(esri_satellite)


m.add_layer(geo_json)
m

Map(center=[34.9, -92.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

#### Create a validation set of 50 barns, with FP/NP markers and a visible boundary layer of the bbox

In [15]:
from ipyleaflet import Map, GeoJSON, WidgetControl, TileLayer, Marker, basemaps, Polygon
from ipywidgets import HTML, Layout, Button, VBox
import json
import random

# Load your GeoJSON data
with open('../output/nebraska_final_data.geojson') as f:
    geojson_data = json.load(f)
for i, feature in enumerate(geojson_data['features']):
    feature['properties']['index'] = i

fp_icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/190/190446.png', icon_size=[25, 25], icon_anchor=[12, 24])
fn_icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/190/190411.png', icon_size=[25, 25], icon_anchor=[12, 24], color='red')

# Generate bbox of 50 barns
def generate_bbox(features):
    attempts = 0
    while attempts < 20:
        random_feature = random.choice(features)
        random_point = random_feature['geometry']['coordinates'][0][0]
        center_lat, center_lon = random_point[1], random_point[0]
        size = 0.01 
        while True:
            lat_min, lat_max = center_lat - size, center_lat + size
            lon_min, lon_max = center_lon - size, center_lon + size
            filtered_features = [
                feature for feature in features if
                lon_min <= feature['geometry']['coordinates'][0][0][0] <= lon_max and
                lat_min <= feature['geometry']['coordinates'][0][0][1] <= lat_max
            ]
            if len(filtered_features) >= 50:
                print(f"BBox coordinates: Lon ({lon_min}, {lon_max}), Lat ({lat_min}, {lat_max})")
                print(f"Total barns in this bbox: {len(filtered_features)}")  
                return filtered_features, (lat_min, lat_max, lon_min, lon_max)
            size += 0.000005 
        attempts += 1
    raise ValueError("Failed to find a suitable bbox with 100 barns")

filtered_features, bbox_coords = generate_bbox(geojson_data['features'])
false_positives = []
# Initialize the map at the center of the bbox
center_lat = (bbox_coords[0] + bbox_coords[1]) / 2
center_lon = (bbox_coords[2] + bbox_coords[3]) / 2
m = Map(
    center=(center_lat, center_lon),
    zoom=12,
    basemap=basemaps.Esri.WorldImagery,
    max_bounds=[(bbox_coords[0], bbox_coords[2]), (bbox_coords[1], bbox_coords[3])]
)

geo_json = GeoJSON(
    data={'type': 'FeatureCollection', 'features': filtered_features},
    style={'color': 'red', 'fillColor': '#ffffff', 'opacity': 1.0, 'fillOpacity': 0.5},
    hover_style={'color': 'green', 'fillOpacity': 0.8}
)

# Setup buttons and info display
fp_button = Button(description='Mark FP', button_style='danger', layout=Layout(width='auto'))
fn_button = Button(description='Mark FN', button_style='success', layout=Layout(width='auto'))
info_display = HTML(value="Select FP or FN and click on the map to place a marker.")

buttons_container = VBox([fp_button, fn_button, info_display])
widget_control = WidgetControl(widget=buttons_container, position='topright')
m.add_control(widget_control)

current_marking = None  # Tracks the current marking type: 'FP' or 'FN'

def set_fp(b):
    global current_marking
    current_marking = 'FP'
    info_display.value = "Marking False Positives. Click on the map."
    geo_json.on_click(handle_fp)

def set_fn(b):
    global current_marking
    current_marking = 'FN'
    info_display.value = "Marking False Negatives. Click on the map."
    m.on_interaction(handle_fn)


def handle_fp(event, feature, **kwargs):

    global m, false_positives
    geometry = feature['geometry']['coordinates'][0][0]
    lon, lat = geometry[:2]
    if current_marking == 'FP':
        # Check if the click is within the bbox before placing a marker
        if bbox_coords[0] <= lat <= bbox_coords[1] and bbox_coords[2] <= lon <= bbox_coords[3]:
            marker = Marker(location=(lat, lon), draggable=False, icon = fp_icon)
            
            false_positives.append(feature['properties']['index'])
            m.add(marker)
            print(feature['properties']['index'])
            print(f"Marked {current_marking} at Latitude: {lat}, Longitude: {lon}")
        
        else:
            info_display.value = "Click is outside the designated area. Try again within the bbox."


def handle_fn(event, **kwargs):
    if kwargs.get('type') == 'click' and current_marking == 'FN':
        latlon = kwargs.get('coordinates')
        lat, lon = latlon[0], latlon[1]
    
    # Check if the click is within the bbox before placing a marker
        if bbox_coords[0] <= lat <= bbox_coords[1] and bbox_coords[2] <= lon <= bbox_coords[3]:
            marker = Marker(location=(lat, lon), draggable=False ,icon = fn_icon)
            
            lat_rounded = round(lat, 2)
            lon_rounded = round(lon, 2)
            info_display.value = f"Marked {current_marking} at Latitude: {lat_rounded}, Longitude: {lon_rounded}"
            m.add(marker)
            print(f"Marked {current_marking} at Latitude: {lat}, Longitude: {lon}")
                    
        else:
            info_display.value = "Click is outside the designated area. Try again within the bbox."




# Add a visible boundary around the bounding box
bbox_polygon = Polygon(
    locations=[
        (bbox_coords[0], bbox_coords[2]),
        (bbox_coords[0], bbox_coords[3]),
        (bbox_coords[1], bbox_coords[3]),
        (bbox_coords[1], bbox_coords[2]),
        (bbox_coords[0], bbox_coords[2])  # Closing the polygon
    ],
    color="blue",
    fill_color="#ffffff",
    opacity=0.5,
    weight=2
)


fp_button.on_click(set_fp)
fn_button.on_click(set_fn)


m.add_layer(bbox_polygon)

# Add GeoJSON layer and satellite imagery
esri_satellite = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
m.add_layer(geo_json)
m.add_layer(esri_satellite)


m


BBox coordinates: Lon (-102.28463097065976, -101.74975097065955), Lat (39.87912555262101, 40.41400555262121)
Total barns in this bbox: 50


Map(center=[40.14656555262111, -102.01719097065966], controls=(ZoomControl(options=['position', 'zoom_in_text'…

1909
Marked FP at Latitude: 40.15419301318038, Longitude: -101.95696779675929
1956
Marked FP at Latitude: 40.159569557512754, Longitude: -102.0065879776989
1962
Marked FP at Latitude: 40.14656555262111, Longitude: -102.01719097065966
Marked FN at Latitude: 40.140816134749436, Longitude: -102.0239262629466
Marked FN at Latitude: 40.13924137258727, Longitude: -102.01259949015032


#### Create a validation set of 50 barns, with FP/NP markers and we can draw a polygon of the bbox (the GeoJSON layer representing the barns remains interactive)

In [5]:
with open('../output/index.txt', 'a') as file:
    for i in false_positives:
        file.write(str(i)+',')